# Implementing a Neural Network
Using the data in the Million Playlist Dataset, I hope to train a neural network to predict good names for a playlist. For now, I will only work with the first file but will later expand to them all (as a test train split)

## Gathering the Data

In [1]:
import json
import pandas as pd

In [2]:
f1 = open('/Users/victoria/Documents/me/Creatify/notebooks/data/mpd.slice.0-999.json')
data = json.load(f1)

# Gather a list of playlists names
playlist_names = []
for i in range(len(data['playlists'])):
    playlist_names.append(data['playlists'][i]['name'])

Starting HERE everything is written for one playlist. Eventually, it will have a larger scope and be within a loop

In [75]:
playlist_tracks = []
for i in range(len(data['playlists'][0]['tracks'])):
    song_id = data['playlists'][0]['tracks'][i]['track_uri']
    print(song_id)
    song_id = song_id.replace("spotify:track:", "")
    print(song_id)
    playlist_tracks.append(song_id)

spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
0UaMYEvWZi0ZqiDOoHU3YI
spotify:track:6I9VzXrHxO9rA9A5euc8Ak
6I9VzXrHxO9rA9A5euc8Ak
spotify:track:0WqIKmW4BTrj3eJFmnCKMv
0WqIKmW4BTrj3eJFmnCKMv
spotify:track:1AWQoqb9bSvzTjaLralEkT
1AWQoqb9bSvzTjaLralEkT
spotify:track:1lzr43nnXAijIGYnCT8M8H
1lzr43nnXAijIGYnCT8M8H
spotify:track:0XUfyU2QviPAs6bxSpXYG4
0XUfyU2QviPAs6bxSpXYG4
spotify:track:68vgtRHr7iZHpzGpon6Jlo
68vgtRHr7iZHpzGpon6Jlo
spotify:track:3BxWKCI06eQ5Od8TY2JBeA
3BxWKCI06eQ5Od8TY2JBeA
spotify:track:7H6ev70Weq6DdpZyyTmUXk
7H6ev70Weq6DdpZyyTmUXk
spotify:track:2PpruBYCo4H7WOBJ7Q2EwM
2PpruBYCo4H7WOBJ7Q2EwM
spotify:track:2gam98EZKrF9XuOkU13ApN
2gam98EZKrF9XuOkU13ApN
spotify:track:4Y45aqo9QMa57rDsAJv40A
4Y45aqo9QMa57rDsAJv40A
spotify:track:1HwpWwa6bnqqRhK8agG4RS
1HwpWwa6bnqqRhK8agG4RS
spotify:track:20ORwCJusz4KS2PbTPVNKo
20ORwCJusz4KS2PbTPVNKo
spotify:track:7k6IzwMGpxnRghE7YosnXT
7k6IzwMGpxnRghE7YosnXT
spotify:track:1Bv0Yl01xBDZD4OQP93fyl
1Bv0Yl01xBDZD4OQP93fyl
spotify:track:4omisSlTk6Dsq2iQD7MA07
4om

In [76]:
# Get the artisits
playlist_artists = []
for i in range(len(data['playlists'][0]['tracks'])):
    artist_id = data['playlists'][0]['tracks'][i]['artist_uri']
    artist_id = artist_id.replace("spotify:artist:", "")
    playlist_artists.append(artist_id)

### Psycho Acoustic Analyzation

In [5]:
from spotipy import oauth2
import re
import credentials
import spotipy

In [104]:
f1 = open('data/mpd.slice.0-999.json')
data = json.load(f1)

In [6]:
# Gain access to mainupation
SCOPE = ('user-read-recently-played,user-library-read,user-read-currently-playing,playlist-read-private,playlist-modify-private,playlist-modify-public,user-read-email,user-modify-playback-state,user-read-private,user-read-playback-state')
sp_oauth = oauth2.SpotifyOAuth(credentials.SPOTIPY_CLIENT_ID,credentials.SPOTIPY_CLIENT_SECRET, credentials.SPOTIPY_REDIRECT_URI ,scope=SCOPE )

#click "Accept" in your browser when the auth window pops up
code = sp_oauth.get_auth_response(open_browser=True)
token = sp_oauth.get_access_token(code)
refresh_token = token['refresh_token']
sp = spotipy.Spotify(auth=token['access_token'])
username = sp.current_user()['id']

/var/folders/pz/lc5f5cgd1m5_ghmjddlz8r600000gn/T/ipykernel_30174/2781803620.py:7: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp_oauth.get_access_token(code)


In [79]:
def analyze_playlist(tracks):
    
    # Create empty dataframe
    playlist_features_list = ["danceability","energy","loudness", "speechiness","instrumentalness","liveness"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    for track in tracks:
        # Get audio features
        audio_features = sp.audio_features(track)
        
        # Concat the dfs
        track_df = pd.DataFrame(audio_features, index=[0])
        partion =  track_df[playlist_features_list]
        playlist_df = pd.concat([playlist_df, partion], axis = 0, ignore_index = True)
        
    return playlist_df

In [80]:
# Get the playlist
playlist_0 = analyze_playlist(playlist_tracks)

### Emotional Statistics

In [81]:
import lyricsgenius
import nltk
import requests
genius = lyricsgenius.Genius('DvaG41Y_l2PyiMbHK8tpKiNzZYZ905GrEo7TTwFCDtqxf7C55K7s36IdKy3AYD9M', verbose=False, retries = 3)

In [82]:
# 2000s, throwback, olides, workout, party, shower
# models = fiction, nytimes, and reddit.
from empath import Empath
lexicon = Empath()
lexicon.create_category("throwback", ["throwback"],model = "reddit")
lexicon.create_category("party", ["party","pregame"],model = "reddit")
lexicon.create_category("workout", ["workout","rap"],model = "reddit")

["throwback", "throwbacks", "throwback", "classic", "80s", "old_school", "throwbacks", "90s", "modern_twist", "iconic", "retro", "Throwback", "retro", "nice_throwback", "90", "harkens", "iconic", "90s", "80s", "2000s", "80s", "harken", "reminiscent", "Reminiscent", "80", "90s", "modern_take", "nostalgic", "90s", "classic", "harkening", "homage", "favorite_era", "stylings", "early_90s", "2000s", "reminiscent", "golden_era", "old-school", "80", "oldschool", "hearkens", "old_style", "esque", "heyday", "80s", "pretty_dope", "mid_90s", "simpler_time", "90", "80", "70s", "late_80s", "themed", "retro_style", "classic_feel", "best_era", "nineties", "late_80s/early_90s", "1990s", "modern_version", "eighties", "classic_look", "nostalgia", "2010s", "way_cooler", "90s", "themed", "different_era", "classic", "2010s", "25th_anniversary", "esque", "late_90s", "80s/90s", "the_early_2000s", "00s", "2000s", "golden_days", "80's/90", "reminds", "nice_nod", "90", "00s", "hay_day", "90s", "80s", "60"]
["pa

In [83]:
track_name = data['playlists'][0]['tracks'][0]['track_name']
artist_name = data['playlists'][0]['tracks'][0]['artist_name']

In [88]:
timeout=20
import numpy as np
emotions_df = pd.DataFrame()
for i in range(len(data['playlists'][0]['tracks'])):
    track_name = data['playlists'][0]['tracks'][i]['track_name']
    artist_name = data['playlists'][0]['tracks'][i]['artist_name']
    try: 
        song = genius.search_song(track_name, artist=artist_name, get_full_info = False)
        if(song != None):
            lyrics = song.lyrics

            # Use the Empath object to analyze the text for different emotions
            emotions = lexicon.analyze(lyrics, normalize=True)
            emotions_df_song = pd.DataFrame(emotions, index=[0])

            emotions_df = pd.concat([emotions_df, emotions_df_song], ignore_index = True)
    except requests.exceptions.Timeout:
        cats = list(lexicon.cats.keys())
        nans = []
        for i in range(len(cats)):
            nans.append(np.nan)
        nones = new_row = pd.Series(nans, index=cats)
        emotions_df = pd.concat([emotions_df, emotions_df_song], ignore_index = True)
        continue

### Combine both togehter
Put the audio features and the emotional features together into one LARGE dataframe!

In [101]:
playlist_data = pd.concat([emotions_df, playlist_0],axis=1, ignore_index = False)

In [99]:
import os

In [103]:
name = "df"+ "0" + ".csv"
directory = 'playlist_data'
filename = os.path.join(directory, name)
playlist_data.to_csv(filename, index=False)

we'll need to eventually store this as a .csv file

## Neural Network Time!

The data we have about each playlist:
1. The audio &  emotion values => playlist_data is for one playlist

What we are trying to predit:
1. The name => playlist_names

In [96]:
import tensorflow as tf
from tensorflow import keras
from torch.utils.data import DataLoader

In [94]:
# Create a dataset object
import os
import pandas as pd

class CustomImageDataset(tf.data.Dataset):
    def __init__(self, names_file, playlist_data_dir):
        self.name_labels = pd.read_csv(names_file) # might change
        self.playlist_data_dir = playlist_data_dir # might change

    def __len__(self):
        return len(self.name_labels)

    def __getitem__(self, idx):
        # we want to get the dataframe correlated to this playlist
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        file = self.playlist_data_dir + "df" + idx + ".csv"
        playlist_data = pd.read_csv(file)
        label = self.name_labels.iloc[idx, 1] # get the label for this 
        return playlist_data, label

In [95]:
# Preparing your data for training with DataLoaders
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

NameError: name 'training_data' is not defined

In [105]:
max = 999000
low = 0
high = 999
while low <= max:
    file_name = "data/mpd.slice." + str(low) + "-" + str(high) + ".json"
    low = low + 1000
    high = high + 1000
    f1 = open(file_name)
    data = json.load(f1)
    print(low)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


KeyboardInterrupt: 